## Import Etmeler Ve Temel Düzenlemeler

In [40]:
import pandas as pd
import os
import numpy as np
Contact_Data = pd.read_csv("https://raw.githubusercontent.com/BiyoinformatikProje/Bulasiciligi-Arttiran-Koronavirus-SARS-CoV-2-Mutasyonlarinin-Tahmini/main/ACE2-RBD_contact_details.txt",sep = " ")
Contact_Data.drop(columns = ["ACE2_ChID","RBD_ChID"],inplace = True)
def getdir(x):
    return os.getcwd() + "\\"+ x

## Tekli Etkileşim Verisinin Çoklu Etkileşim Verisine Çevrilmesi

In [41]:
temp = []
for rbd in Contact_Data["RBD_ResNo"].unique():
    current = {}
    for ace2 in Contact_Data[Contact_Data["RBD_ResNo"] == rbd]["ACE2_ResNo"]:
        if ace2 in current:
            current[ace2] +=1
        else:
            current[ace2] = 1
    temp.append(current)
rbd = Contact_Data["RBD_ResNo"].unique()
Multi_Contact_Data = pd.DataFrame({"RBD_ResNo":rbd,"ACE2_Interaction_ResNo":temp })
Multi_Contact_Data

,RBD_ResNo,ACE2_Interaction_ResNo
0,475,"{19: 1, 24: 7, 27: 3}"
1,476,{24: 5}
2,486,"{24: 2, 79: 7, 82: 15, 83: 20}"
3,487,"{24: 17, 83: 12}"
4,489,"{24: 1, 27: 14, 28: 14, 31: 11, 83: 3}"
5,456,"{27: 12, 30: 8, 31: 7}"
6,473,{27: 2}
7,417,"{30: 7, 34: 1}"
8,455,"{30: 7, 31: 4, 34: 15}"
9,484,{31: 2}


## Etkileşim Dosyalarından Rezidü Numarasına Denk Gelen Aminoasitlerin Grafiğinin Çıkarılması

### ACE2 Residü Numarasına Karşılık Gelen Aminoasitlerin Bulunduğu Veri Dosyasını Sadeleştirme ve Python'a Uygun Hale Getirme

In [42]:
acids = {"ARG":"R","HIS":"H","LYS":"K","ASP":"D","GLU":"E","SER":"S","THR":"T","ASN":"N","GLN":"Q","CYS":"C","SEC":"U","GLY":"G","PRO":"P","ALA":"A","ILE":"I","LEU":"L","MET":"M","PHE":"F","TRP":"W","TYR":"Y","VAL":"V"}
df = pd.DataFrame(columns = ["AMINO_ACID","CHAIN","RESIDUE_NO"])
cwd = os.getcwd()
with open(cwd + "\Write.txt","w") as aa:
    with open(cwd + "\\6m0j.pdb","r") as pdb:
        for line in pdb:
            new = (line[17:26].replace("  "," ")).replace(" ",",")
            aa.write(new + "\n")
newdata = pd.read_csv(getdir("Write.txt"))
newdata.columns = ["AMINO_ACID","CHAIN","RESIDUE_NO"]
newdata.to_csv("RBD_CONTACT_DATA.txt",index = False)
os.remove(cwd + "\Write.txt")
newdata

,AMINO_ACID,CHAIN,RESIDUE_NO
0,SER,A,19
1,SER,A,19
2,SER,A,19
3,SER,A,19
4,SER,A,19
...,...,...,...
6413,CYS,E,525
6414,GLY,E,526
6415,GLY,E,526
6416,GLY,E,526


### Bu Veriyi ACE2 ve RBD Zincirlerine Ayırma

In [43]:
last = 0
rbdres = {}
rbdchain = newdata[newdata["CHAIN"] == "E"]
for i in rbdchain.iloc:
    if i["RESIDUE_NO"] != last:
        last = i["RESIDUE_NO"]
        rbdres[last] = acids[i["AMINO_ACID"]]
ace2chain = newdata[newdata["CHAIN"] == "A"]
ace2res = {}
for i in ace2chain.iloc:
    if i["RESIDUE_NO"] != last:
        last = i["RESIDUE_NO"]
        ace2res[last] = acids[i["AMINO_ACID"]]

## Rezidü Numaralarının Aminoasit Karşılıklarını Oluşturulan Grafikten Çekerek Veriye İşleme

### ACE2_Interaction_ResNo Sütunundaki Rezidü Numaralarının Aminoasit Koduna Çevrilmesi

In [44]:
pozitions = []
interactions = []
lettercode = []
pozitions_sum = []
interactions_sum = []
lettercode_sum = []
total = []
for i in Multi_Contact_Data["ACE2_Interaction_ResNo"]:
    for j in i:
        pozitions.append(j)
        interactions.append(i[j])
        lettercode.append(ace2res[j])
    x = len(pozitions)
    if x < 5:
        for i in range(5-x):
            pozitions.append(0)
            interactions.append(0)
            lettercode.append(0)
    pozitions_sum.append(pozitions.copy())
    interactions_sum.append(interactions.copy())
    lettercode_sum.append(lettercode.copy())
    pozitions.clear()
    interactions.clear()
    lettercode.clear()
Multi_Contact_Data["Positions"] = pozitions_sum 
Multi_Contact_Data["Interaction_Amount"] = interactions_sum 
Multi_Contact_Data["Lettercode"] = lettercode_sum
Multi_Contact_Data


,RBD_ResNo,ACE2_Interaction_ResNo,Positions,Interaction_Amount,Lettercode
0,475,"{19: 1, 24: 7, 27: 3}","[19, 24, 27, 0, 0]","[1, 7, 3, 0, 0]","[S, Q, T, 0, 0]"
1,476,{24: 5},"[24, 0, 0, 0, 0]","[5, 0, 0, 0, 0]","[Q, 0, 0, 0, 0]"
2,486,"{24: 2, 79: 7, 82: 15, 83: 20}","[24, 79, 82, 83, 0]","[2, 7, 15, 20, 0]","[Q, L, M, Y, 0]"
3,487,"{24: 17, 83: 12}","[24, 83, 0, 0, 0]","[17, 12, 0, 0, 0]","[Q, Y, 0, 0, 0]"
4,489,"{24: 1, 27: 14, 28: 14, 31: 11, 83: 3}","[24, 27, 28, 31, 83]","[1, 14, 14, 11, 3]","[Q, T, F, K, Y]"
5,456,"{27: 12, 30: 8, 31: 7}","[27, 30, 31, 0, 0]","[12, 8, 7, 0, 0]","[T, D, K, 0, 0]"
6,473,{27: 2},"[27, 0, 0, 0, 0]","[2, 0, 0, 0, 0]","[T, 0, 0, 0, 0]"
7,417,"{30: 7, 34: 1}","[30, 34, 0, 0, 0]","[7, 1, 0, 0, 0]","[D, H, 0, 0, 0]"
8,455,"{30: 7, 31: 4, 34: 15}","[30, 31, 34, 0, 0]","[7, 4, 15, 0, 0]","[D, K, H, 0, 0]"
9,484,{31: 2},"[31, 0, 0, 0, 0]","[2, 0, 0, 0, 0]","[K, 0, 0, 0, 0]"


### ACE2_ResNo Sütunundaki Rezidü Numaralarının Aminoasit Kodlarına Çevrilmesi

In [45]:
rbd_acids = []
for i in range(len(Multi_Contact_Data)):
    rbd_acids.append(rbdres[Multi_Contact_Data["RBD_ResNo"].iloc[i]])
Multi_Contact_Data["RBD_Amino_Acids"] = rbd_acids 
Multi_Contact_Data

,RBD_ResNo,ACE2_Interaction_ResNo,Positions,Interaction_Amount,Lettercode,RBD_Amino_Acids
0,475,"{19: 1, 24: 7, 27: 3}","[19, 24, 27, 0, 0]","[1, 7, 3, 0, 0]","[S, Q, T, 0, 0]",A
1,476,{24: 5},"[24, 0, 0, 0, 0]","[5, 0, 0, 0, 0]","[Q, 0, 0, 0, 0]",G
2,486,"{24: 2, 79: 7, 82: 15, 83: 20}","[24, 79, 82, 83, 0]","[2, 7, 15, 20, 0]","[Q, L, M, Y, 0]",F
3,487,"{24: 17, 83: 12}","[24, 83, 0, 0, 0]","[17, 12, 0, 0, 0]","[Q, Y, 0, 0, 0]",N
4,489,"{24: 1, 27: 14, 28: 14, 31: 11, 83: 3}","[24, 27, 28, 31, 83]","[1, 14, 14, 11, 3]","[Q, T, F, K, Y]",Y
5,456,"{27: 12, 30: 8, 31: 7}","[27, 30, 31, 0, 0]","[12, 8, 7, 0, 0]","[T, D, K, 0, 0]",F
6,473,{27: 2},"[27, 0, 0, 0, 0]","[2, 0, 0, 0, 0]","[T, 0, 0, 0, 0]",Y
7,417,"{30: 7, 34: 1}","[30, 34, 0, 0, 0]","[7, 1, 0, 0, 0]","[D, H, 0, 0, 0]",K
8,455,"{30: 7, 31: 4, 34: 15}","[30, 31, 34, 0, 0]","[7, 4, 15, 0, 0]","[D, K, H, 0, 0]",L
9,484,{31: 2},"[31, 0, 0, 0, 0]","[2, 0, 0, 0, 0]","[K, 0, 0, 0, 0]",E


## ACE2-RBD İnteraksiyon Verisinin Özellikler İle Birleştirilmesi

### Özellik Tablosunun Oluşturulması

In [46]:
aminoasitler = ["H","R","K","I","F","L","W","A","M","P","C","N","V","G","S","Q","Y","D","E","T"]
polarite = {aminoasitler[i]: [0,0,0,1,1,1,1,1,1,1,2,2,1,2,2,2,2,3,3,2][i] for i in range(len(aminoasitler))}
hacim = {aminoasitler[i]: [153.2,173.4,168.6,166.7,189.9,166.7,227.8,88.6,162.9,112.7,108.5,114.1,140.0,60.1,89.0,143.8,193.6,111.1,138.4,116.1][i] for i in range(len(aminoasitler))}
features = pd.read_csv("https://raw.githubusercontent.com/BiyoinformatikProje/Bulasiciligi-Arttiran-Koronavirus-SARS-CoV-2-Mutasyonlarinin-Tahmini/main/Ozellikler.csv")
aminoacids= ["R","H","K","D","E","S","T","N","Q","C","U","G","P","A","V","I","L","M","F","Y","W"]
ring     = {aminoacids[i]: [0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,2][i] for i in range(len(aminoacids))}
dbond    = {aminoacids[i]: [2,1,1,2,2,1,1,2,2,1,1,1,1,1,1,1,1,1,1,1,1][i] for i in range(len(aminoacids))}
carbon   = {aminoacids[i]: [6,3,6,4,5,3,4,4,5,3,3,2,5,3,5,6,6,5,3,3,3][i] for i in range(len(aminoacids))}
oxygen   = {aminoacids[i]: [2,2,2,4,4,3,2,3,3,2,2,2,2,2,2,2,2,2,2,2,2][i] for i in range(len(aminoacids))}
hydrogen = {aminoacids[i]: [14,7,14,7,9,7,9,8,10,7,7,5,9,7,11,13,13,11,11,11,12][i] for i in range(len(aminoacids))}
polarite2 = []
hacim2 = []
ring2 = []
dbond2 = []
carbon2 = []
oxygen2 = []
hydrogen2 = []
for i in features["Amino acid code"]:
    polarite2.append(polarite[i])
    hacim2.append(hacim[i])
    ring2.append(ring[i])
    dbond2.append(dbond[i])
    carbon2.append(carbon[i])
    oxygen2.append(oxygen[i])
    hydrogen2.append(hydrogen[i])
features["Double Bond Number"] = dbond2 
features["Ring Number"] = ring2 
features["Volume"] = hacim2 
features["Oxygen Number"] = oxygen2 
features["Hydrogen Number"] = hydrogen2 
features["Polarity"] = polarite2 
features["Carbon Number"] = carbon2 
def get_feature(amino_acid,feature):
    return features[features["Amino acid code"] == amino_acid][feature].iloc[0]

### Özelliklerin RBD-ACE2 İnteraksiyon Verisiyle Birleştirilmesi

In [47]:
ring     = []
dbond    = [] 
carbon   = [] 
oxygen   = []
hydrogen = []
molecularmass = []
hydropathy = []
volume = []
polarity = []
ringtots   = 0
dbondtots = 0
carbontots= 0
oxygentots= 0
hydrogentots  = 0
molecularmasstots = 0
hydropathytots = 0
volumetots = 0
polaritytots = 0
for i in Multi_Contact_Data["ACE2_Interaction_ResNo"]:
    for j in i:
        ringtots+= get_feature(ace2res[j],"Ring Number") * i[j]
        dbondtots+= get_feature(ace2res[j],"Double Bond Number") * i[j]
        carbontots+= get_feature(ace2res[j],"Carbon Number") * i[j]
        oxygentots+= get_feature(ace2res[j],"Oxygen Number") * i[j]
        hydrogentots+= get_feature(ace2res[j],"Hydrogen Number") * i[j]
        molecularmasstots += get_feature(ace2res[j],"Molecular Mass (Da)") * i[j]
        hydropathytots += get_feature(ace2res[j],"Hydropathy index") * i[j]
        volumetots += get_feature(ace2res[j],"Volume") * i[j]
        polaritytots += get_feature(ace2res[j],"Polarity") * i[j]
    ring.append(ringtots)
    dbond.append(dbondtots)
    carbon.append(carbontots)
    oxygen.append(oxygentots)
    hydrogen.append(hydrogentots)
    molecularmass.append(molecularmasstots)
    hydropathy.append(hydropathytots)
    volume.append(volumetots)
    polarity.append(polaritytots)
    ringtots   = 0
    dbondtots = 0
    carbontots= 0
    oxygentots= 0
    hydrogentots  = 0
    molecularmasstots = 0
    hydropathytots = 0
    volumetots = 0
    polaritytots = 0
Multi_Contact_Data["Total Ring Number"] = ring
Multi_Contact_Data["Total Double Bond Number"] = dbond 
Multi_Contact_Data["Total Carbon Number"] = carbon 
Multi_Contact_Data["Total Oxygen Number"] = oxygen 
Multi_Contact_Data["Total Hydrogen Number"] = hydrogen 
Multi_Contact_Data["Total Molecular Mass"] = molecularmass 
Multi_Contact_Data["Total Hydropathy"] = hydropathy 
Multi_Contact_Data["Total Volume"] = volume 
Multi_Contact_Data["Total Polarity"] = polarity
Multi_Contact_Data


,RBD_ResNo,ACE2_Interaction_ResNo,Positions,Interaction_Amount,Lettercode,RBD_Amino_Acids,Total Ring Number,Total Double Bond Number,Total Carbon Number,Total Oxygen Number,Total Hydrogen Number,Total Molecular Mass,Total Hydropathy,Total Volume,Total Polarity
0,475,"{19: 1, 24: 7, 27: 3}","[19, 24, 27, 0, 0]","[1, 7, 3, 0, 0]","[S, Q, T, 0, 0]",A,0,18,50,30,104,1484.0,-27.4,1443.9,22
1,476,{24: 5},"[24, 0, 0, 0, 0]","[5, 0, 0, 0, 0]","[Q, 0, 0, 0, 0]",G,0,10,25,15,50,730.0,-17.5,719.0,10
2,486,"{24: 2, 79: 7, 82: 15, 83: 20}","[24, 79, 82, 83, 0]","[2, 7, 15, 20, 0]","[Q, L, M, Y, 0]",F,20,46,187,90,496,7064.0,22.1,7770.0,66
3,487,"{24: 17, 83: 12}","[24, 83, 0, 0, 0]","[17, 12, 0, 0, 0]","[Q, Y, 0, 0, 0]",N,12,46,121,75,302,4654.0,-75.1,4767.8,58
4,489,"{24: 1, 27: 14, 28: 14, 31: 11, 83: 3}","[24, 27, 28, 31, 83]","[1, 14, 14, 11, 3]","[Q, T, F, K, Y]",Y,17,44,178,87,477,6271.0,-20.9,6863.2,50
5,456,"{27: 12, 30: 8, 31: 7}","[27, 30, 31, 0, 0]","[12, 8, 7, 0, 0]","[T, D, K, 0, 0]",F,0,35,122,70,262,3514.0,-63.7,3462.2,48
6,473,{27: 2},"[27, 0, 0, 0, 0]","[2, 0, 0, 0, 0]","[T, 0, 0, 0, 0]",Y,0,2,8,4,18,238.0,-1.4,232.2,4
7,417,"{30: 7, 34: 1}","[30, 34, 0, 0, 0]","[7, 1, 0, 0, 0]","[D, H, 0, 0, 0]",K,1,15,31,30,56,1086.0,-27.7,930.9,21
8,455,"{30: 7, 31: 4, 34: 15}","[30, 31, 34, 0, 0]","[7, 4, 15, 0, 0]","[D, K, H, 0, 0]",L,15,33,97,66,210,3840.0,-88.1,3750.1,21
9,484,{31: 2},"[31, 0, 0, 0, 0]","[2, 0, 0, 0, 0]","[K, 0, 0, 0, 0]",E,0,2,12,4,28,292.0,-7.8,337.2,0


### Satır İsimlerinin Sırasının Düzenlenmesi 

In [48]:
cols = Multi_Contact_Data.columns
cols = cols.insert(1,"RBD_Amino_Acids")
cols = cols.delete(5)
Multi_Contact_Data = Multi_Contact_Data[cols]
Multi_Contact_Data 

,RBD_ResNo,RBD_Amino_Acids,ACE2_Interaction_ResNo,Positions,Interaction_Amount,RBD_Amino_Acids,Total Ring Number,Total Double Bond Number,Total Carbon Number,Total Oxygen Number,Total Hydrogen Number,Total Molecular Mass,Total Hydropathy,Total Volume,Total Polarity
0,475,A,"{19: 1, 24: 7, 27: 3}","[19, 24, 27, 0, 0]","[1, 7, 3, 0, 0]",A,0,18,50,30,104,1484.0,-27.4,1443.9,22
1,476,G,{24: 5},"[24, 0, 0, 0, 0]","[5, 0, 0, 0, 0]",G,0,10,25,15,50,730.0,-17.5,719.0,10
2,486,F,"{24: 2, 79: 7, 82: 15, 83: 20}","[24, 79, 82, 83, 0]","[2, 7, 15, 20, 0]",F,20,46,187,90,496,7064.0,22.1,7770.0,66
3,487,N,"{24: 17, 83: 12}","[24, 83, 0, 0, 0]","[17, 12, 0, 0, 0]",N,12,46,121,75,302,4654.0,-75.1,4767.8,58
4,489,Y,"{24: 1, 27: 14, 28: 14, 31: 11, 83: 3}","[24, 27, 28, 31, 83]","[1, 14, 14, 11, 3]",Y,17,44,178,87,477,6271.0,-20.9,6863.2,50
5,456,F,"{27: 12, 30: 8, 31: 7}","[27, 30, 31, 0, 0]","[12, 8, 7, 0, 0]",F,0,35,122,70,262,3514.0,-63.7,3462.2,48
6,473,Y,{27: 2},"[27, 0, 0, 0, 0]","[2, 0, 0, 0, 0]",Y,0,2,8,4,18,238.0,-1.4,232.2,4
7,417,K,"{30: 7, 34: 1}","[30, 34, 0, 0, 0]","[7, 1, 0, 0, 0]",K,1,15,31,30,56,1086.0,-27.7,930.9,21
8,455,L,"{30: 7, 31: 4, 34: 15}","[30, 31, 34, 0, 0]","[7, 4, 15, 0, 0]",L,15,33,97,66,210,3840.0,-88.1,3750.1,21
9,484,E,{31: 2},"[31, 0, 0, 0, 0]","[2, 0, 0, 0, 0]",E,0,2,12,4,28,292.0,-7.8,337.2,0


### Verinin Analizde Kullanılmak Üzere .csv Formatına Çevrilmesi


In [49]:
Multi_Contact_Data.to_csv("Multi_Contact_Data.csv",index = False)